In [1]:
cd /content/drive/MyDrive/PYTORCH/Practices

/content/drive/MyDrive/PYTORCH/Practices


In [2]:
import torch
from torch import nn 
import torchvision 
import torch.nn.functional as F
import torchvision.transforms as transforms 
import numpy as np 
import matplotlib.pyplot as plt 
from datetime import datetime

In [3]:
train_dataset = torchvision.datasets.MNIST(root=".", 
                                  train=True, 
                                  transform=transforms.ToTensor(),                          
                                  download=True)
test_dataset = torchvision.datasets.MNIST(root=".", 
                                  train=False, 
                                  transform=transforms.ToTensor(),                          
                                  download=True)

In [19]:
batch_size = 100
learning_rate = 0.001
num_classes = 10
input_size = 28
sequence_length = 28 
hidden_size = 128 
num_layers = 2
num_epochs = 2

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [7]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [15]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
    super(RNN, self).__init__()

    self.input_size = input_size

    self.hidden_size = hidden_size

    self.num_layers = num_layers

    self.num_classes = num_classes
    #shape of x->  batch_size, sequence,  input size
    self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
    
    self.fc = nn.Linear(hidden_size, num_classes)


  def forward(self, x):
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
    out, h_n = self.rnn(x, h0)
    #shape output--> batch_size, sequence length, hidden size 
    # out (N, 28, 128)
    #(N, 128)
    out = out[:, -1, :]

    out = self.fc(out)

    return out




In [16]:
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [18]:
print(model) 

RNN(
  (rnn): RNN(28, 128, num_layers=2, batch_first=True)
  (fc): Linear(in_features=128, out_features=10, bias=True)
)


In [20]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # origin shape: [N, 1, 28, 28]
        # resized: [N, 28, 28]
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')


Epoch [1/2], Step [100/600], Loss: 0.9340
Epoch [1/2], Step [200/600], Loss: 0.6535
Epoch [1/2], Step [300/600], Loss: 0.7261
Epoch [1/2], Step [400/600], Loss: 0.3903
Epoch [1/2], Step [500/600], Loss: 0.2809
Epoch [1/2], Step [600/600], Loss: 0.4118
Epoch [2/2], Step [100/600], Loss: 0.1574
Epoch [2/2], Step [200/600], Loss: 0.1281
Epoch [2/2], Step [300/600], Loss: 0.2667
Epoch [2/2], Step [400/600], Loss: 0.1759
Epoch [2/2], Step [500/600], Loss: 0.1247
Epoch [2/2], Step [600/600], Loss: 0.1134


In [21]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the 10000 test images: {acc} %')

Accuracy of the network on the 10000 test images: 94.48 %
